#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계2 : 모델링

## 0.미션

* 다음 세가지 챗봇을 만들고 비교해 봅시다.
* 챗봇1. Word2Vec 임베딩 벡터 기반 머신러닝 분류 모델링
    * Word2Vec 모델을 만들고 임베딩 벡터를 생성합니다.
    * 임베딩 벡터를 이용하여 intent를 분류하는 모델링을 수행합니다.
        * 이때, LightGBM을 추천하지만, 다른 알고리즘을 이용할수 있습니다.
    * 예측된 intent의 답변 중 임의의 하나를 선정하여 출력합니다.
* 챗봇2. 단계별 모델링1
    * 1단계 : type(일상대화 0, 에이블스쿨Q&A 1) 분류 모델 만들기
        * Embedding + LSTM 모델링
    * 2단계 : 사전학습된 Word2Vec 모델을 로딩하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기
* 챗봇3. 단계별 모델링2
    * 1단계 : 챗봇2의 1단계 모델을 그대로 활용
    * 2단계 : FastText 모델 생성하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기

* 챗봇3개에 대해서 몇가지 질문을 입력하고 각각의 답변을 비교해 봅시다.


## 1.환경준비

### (1)라이브러리 설치

#### 1) gensim 3.8.3 설치

In [23]:
#gensim은 자연어 처리를 위한 오픈소스 라이브러리입니다. 토픽 모델링, 워드 임베딩 등 다양한 자연어 처리 기능을 제공
# 현재 4.x 버전이 최신이지만, 3.8.3 버전으로 진행
!pip uninstall gensim
!pip install gensim==3.8.3

Found existing installation: gensim 4.3.1
Uninstalling gensim-4.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/gensim-4.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/gensim/*
Proceed (Y/n)? Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 664, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_uninstall.py", line 367, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_uninstall.py", line 407, in _allowed_to_proceed
    return ask

* [코랩] 위 라이브러리 설치후 런타임 재시작 필요!

#### 2) 형태소 분석을 위한 라이브러리

In [1]:
# mecab 설치를 위한 관련 패키지 설치
!apt-get install curl git
!apt-get install build-essential
!apt-get install cmake
!apt-get install g++
!apt-get install flex
!apt-get install bison
!apt-get install python-dev
!pip install cython
!pip install mecab-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.18).
git is already the newest version (1:2.25.1-1ubuntu3.11).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Rea

In [2]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu: linux, mac os 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


### (2) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 필요하다고 판단되는 라이브러리를 추가하세요.
import os

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import * 

import tensorflow as tf
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics.pairwise import cosine_similarity



* 형태소 분석을 위한 함수를 제공합니다.

In [4]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        # "kkma":
        tokenizer = Kkma()
        
    return tokenizer

In [5]:
# 형태소 분석을 수행하는 함수

def tokenize(tokenizer_name, original_sent, nouns=False):
    # 미리 정의된 몇 가지 tokenizer 중 하나를 선택
    tokenizer = get_tokenizer(tokenizer_name)

    # tokenizer를 이용하여 original_sent를 토큰화하여 tokenized_sent에 저장하고, 이를 반환합니다.
    sentence = original_sent.replace('\n', '').strip()
    if nouns:       
        # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.nouns(sentence)
    else:
        tokens = tokenizer.morphs(sentence)
    tokenized_sent = ' '.join(tokens)
    
    return tokenized_sent

### (3) 데이터 로딩
* 전처리 단계에서 생성한 데이터들을 로딩합니다.
    * train, test
    * 형태소분석 결과 데이터 : clean_train_questions, clean_test_questions

* 구글 드라이브 연결

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path = '/content/drive/MyDrive/0417_미니프로젝트 6차_실습자료/'

* 저장된 .pkl 파일들을 불러옵니다.
* 불러 온 후에는 shape를 확인해 봅시다.

In [8]:
train = joblib.load(path + 'train.pkl')
test = joblib.load(path + 'test.pkl')
clean_test_questions = joblib.load(path + 'clean_test_questions.pkl')
clean_train_questions = joblib.load(path + 'clean_train_questions.pkl')

In [9]:
train.shape, test.shape, clean_test_questions.shape, clean_train_questions.shape

((1113, 4), (106, 4), (106,), (1113,))

## 2.챗봇1

* **상세요구사항**
    * Word2Vec을 활용한 LightGBM 모델링(intent 분류)
        * Word2Vec을 이용하여 임베딩벡터 생성하기
            * Word Embedding으로 문장벡터 구하기
        * 임베딩 벡터를 이용하여 ML기반 모델링 수행하기
            * LightGBM 권장(다른 알고리즘을 이용할수 있습니다.)
    * 챗봇 : 모델의 예측결과(intent)에 따라 답변하는 챗봇 만들기
        * 질문을 입력받아, 답변하는 함수 생성

### (1) Word2Vec을 이용하여 임베딩벡터 생성하기
* 'mecab' 형태소 분석기를 이용하여 문장을 tokenize
    * Word2Vec 모델을 만들기 위해서 입력 데이터는 리스트 형태여야 합니다.
    * 그래서 다시 리스트로 저장되도록 토크나이즈 해 봅시다.
* Word Embedding으로 문장벡터를 생성합니다.
    * 먼저 Word2Vec 모델을 만들고, train의 질문들을 문장벡터로 만듭시다.


#### 1) 'mecab' 형태소 분석기를 이용하여 문장을 tokenize

In [10]:
#train의 질문들을 리스트로 뽑고
raw_sentence = list(train['Q'])

# train_sentences 리스트를 초기화
train_sentence = []

# raw_swmtemce에서 하나씩 가져와서 토큰화하여 train_sentence에 추가한다
for sent in raw_sentence:
    # tokenize함수를 통해 mecab형태소 분석기를 이용하여 문장을 토큰화한다
    # split함수를 이용하여 토큰화된 단어들을 리스트로 만들어 추가한다
    train_sentence.append(tokenize('mecab', sent).split(' '))

print(train_sentence[:5])

[['떨어뜨려서', '핸드폰', '액정', '나갔', '어'], ['핸드폰', '떨어뜨려서', '고장', '났', '나', '봐'], ['노트북', '이', '작동', '이', '안', '되', '네'], ['노트북', '키보드', '가', '안', '먹히', '네'], ['노트북', '전원', '이', '안', '들어와', '.']]


#### 2) Word Embedding으로 문장벡터 구하기
* Word2Vec
    * 위에서 저장한 입력 데이터를 사용하여 Word2Vec 모델이 생성
    * 모델은 size(단어 벡터의 차원), 
    * window(컨텍스트 창의 크기), 
    * max_vocab_size(고려할 최대 어휘 크기), 
    * min_count(포함할 단어의 최소 빈도)와 같은 특정 하이퍼파라미터로 훈련됩니다.
    * sg : 사용할 훈련 알고리즘 - 1은 skip-gram, 0은 CBOW )

In [11]:
# Word2Vec은 임베딩할 때, 워드 벡터의 차원 수를 지정하는 변수로 100 지정되어 있음
SIZE = 100
# Word2Vec 모델에서 한 단어의 좌우에 있는 최대 단어 개수를 지정하는 변수로 3 지정되어 있음
WINDOW = 3
# Word2Vec 모델에 의해 무시될 최소 단어 빈도 수를 지정하는 변수로 해당 값보다 적게 나타나는 단어 수
MIN_COUNT = 5

In [12]:
from gensim.models import Word2Vec

# Word2Vec 모델 생성
wv_model = Word2Vec(train_sentence,        #학습할 문장
                    vector_size=SIZE,             #워드 임베딩 벡터의 크기
                    window=WINDOW,         #문맥 윈도우 크기
                    max_vocab_size=500,    #학습에 사용될 최대 단어 수
                    min_count=MIN_COUNT,   #학습에 사용할 최소 단어 빈도 수
                    workers=4,              #학습을 위한 프로세스 수
                    epochs=10,               #학습 횟수
                    sg=1)                  #skip-Gram방식사용

* Word2Vec 모델로부터 데이터를 벡터화하기 위한 함수 생성

In [13]:
# Word2Vec 모델로부터 하나의 문장을 벡터화 시키는 함수
def get_sent_embedding(model, embedding_size, tokenized_words):
    # 임베딩 벡터를 0으로 초기화
    feature_vec = np.zeros((embedding_size,), dtype='float32')
    # 단어 개수 초기화
    n_words = 0
    # 모델 단어 집합 생성
    index2word_set = set(model.wv.index2word)
    # 문장의 단어들을 하나씩 반복
    for word in tokenized_words:
        # 모델 단어 집합에 해당하는 단어일 경우에만
        if word in index2word_set:
            # 단어 개수 1 증가
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model[word])
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [14]:
# 문장벡터 데이터 셋 만들기
def get_dataset(sentences, model, num_features):
    dataset = list()

    # 각 문장을 벡터화해서 리스트에 저장
    for sent in sentences:
        dataset.append(get_sent_embedding(model, num_features, sent))

    # 리스트를 numpy 배열로 변환하여 반환
    sent_embedding_vectors = np.stack(dataset)
    
    return sent_embedding_vectors

* 이제 학습데이터의 Q를 Word2Vec 모델을 사용하여 벡터화 합니다.

In [15]:
# 학습 데이터의 문장들을 Word2Vec 모델을 사용하여 벡터화
train_data_vecs = get_dataset(train_sentence, wv_model, SIZE)
train_data_vecs.shape


AttributeError: ignored

* 훈련된 Word2Vec 모델을 사용하여 문장 목록에 대한 문장 임베딩을 생성하고 이를 2차원 numpy 배열에 저장합니다. 
* 그런 다음 이러한 임베딩을 다양한 기계 학습 모델의 입력 기능으로 사용할 수 있습니다

### (2) 분류 모델링
* 데이터 분할
    * x, y
        * x : 이전 단계에서 저장된 임베딩벡터(train_data_vecs)
        * y : intent 값들
    * train, val
        * train_test_split 활용
* 머신러닝 모델링
    * lightGBM, RandomForest 등을 활용하여 학습
    * 필요하다면 hyper parameter 튜닝을 시도해도 좋습니다.
* validation set으로 검증해 봅시다.

In [ ]:
# X와 y 데이터 분리
x = train_data_vecs
y = np.array(list(train['intent']))
# Train-Test split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

* 모델1

In [ ]:
lgbm = LGBMClassifier(n_estimators=200, objective='multiclass')

lgbm.fit(x_train, y_train)

In [ ]:
pred1 = lgbm.predict(x_val)
accuracy_score(y_val, pred1)

* 모델2

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)

rf.fit(x_train, y_train)

In [ ]:
pred2 = rf.predict(x_val)
accuracy_score(y_val, pred2)

모델저장


In [ ]:
import joblib

joblib.dump(lgbm, path+'lgbm_classifier.pkl')

### (3) 챗봇 구축

* **상세요구사항**
    * 챗봇 flow : input 질문 -> 분류 모델로 intent 예측 --> intent에 해당하는 답변 출력
        * 하나의 intent 에는 여러 답변이 있습니다. 이중 한가지를 랜덤하게 선택합니다.

#### 1) 데이터 중 하나에 대해서 테스트

In [ ]:
n = 1
test['Q'].loc[n]

In [ ]:
#입력문장 벡터화
test_data_vec = get_dataset([test['Q'].iloc[n]], wv_model, SIZE)

In [ ]:
#분류 모델을 이용하여 intent 예측
p1 = lgbm.predict(test_data_vec)[0]
p2 = rf.predict(test_data_vec)[0]
p1, p2

In [ ]:
print('질문: ', test['Q'].iloc[n])
answer1 = train.loc[train['intent']==p1, 'A'].unique()
print('답변1: ', np.random.choice(answer1))

answer2 = train.loc[train['intent']==p2, 'A'].unique()
print('답변2: ', np.random.choice(answer2))

#### 2) 챗봇 함수 만들기
* 테스트 코드를 바탕으로 질문을 받아 답변을 하는 함수를 생성합시다.
* 성능이 좋은 모델 사용. 

In [ ]:
def get_answer1(question): 
    test_data_vec = get_dataset([question], wv_model, SIZE)
    pred = rf.predict(test_data_vec)[0]
    answers = train.loc[train['intent']==pred, 'A'].unique()
    answer = np.random.choice(answers)

    return answer, pred

In [ ]:
question = '4년제 졸업자만 지원 가능한가요?'
get_answer1(question)

('학교 사정으로 졸업예정증명서 발급이 불가능한 경우 수료증명서 또는 재학증명서를 제출하여 주시기 바라며, 추후 졸업예정증명서 발급 가능 시점에 제출 요청을 드릴 수 있습니다.\n대학원 재학 중으로 성적증명서가 없으신 경우, 최종학력은 대학교로 입력하여 주시고 졸업(예정)증명서 및 성적증명서 제출란에는 재학증명서를 제출하여 주시기 바랍니다.\n국외 대학의 경우, 졸업을 확인할 수 있는 서류인 공식 졸업장(예: Diploma)과, 4년간의 이수 과목과 성적을 확인할 수 있는 성적표(예: Transcript)를 제출하시면 됩니다.',
 39)

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

In [ ]:
ct = 0
for tdx, row in test.iterrows():
    print('질문: ', row['Q'])
    answer, intent_idx = get_answer1(row['Q'])
    print('예상 답변: ', answer)
    print('-'*200) 
    if intent_idx == row['intent']:
        ct += 1
print(ct, test.shape[0])

## 3.챗봇2

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : type을 0과 1로 분류하는 모델 생성(Embedding + LSTM 모델)
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링(LSTM)
- LSTM

#### 1) 데이터 준비
* 학습용 데이터를 만들어 봅시다.
    * 시작 데이터 : clean_train_questions, clean_test_questions
    * 각 토큰에 인덱스를 부여하는 토크나이저를 만들고 적용
        * from tensorflow.keras.preprocessing.text import Tokenizer 를 사용
    * 문장별 길이에 대한 분포를 확인하고 적절하게 정의.

In [ ]:
# 각각의 토큰에 인덱스 부여하는 토크나이저 선언
tokenizer = Tokenizer()

# .fit_on_tests 이용하여 토크나이저 만들기
tokenizer.fit_on_textx(clean_train_questions)

In [ ]:
# 전체 토큰의 수 확인
import json
print(json.loads(tokenizer.to_json())['config']['word_index'])
len(tokenizer.word_index.keys())

In [ ]:
# 전체 토큰의 수로 vocab_size 지정
vocab_size = len(tokenizer.word_index.keys())
tokenizer = Tokenizer(vocab_size, oov_token = '<ooV>')

# fit_on_texts을 위에서 한번만 해도 되지만, vocab 사이즈를 확인하고 줄이거나 하는 시도를 할 수도 있기에 다시 수행
tokenizer.fit_on_texts(clean_train_questions)

# .texts_to_sequences : 토크나이즈 된 데이터를 가지고 모두 시퀀스로 변환
x_train = tokenizer.texts_to_sequences(clean_train_questions)
x_val = tokenizer.texts_to_sequences(clean_test_qeustions)

In [ ]:
# 전체 토큰의 수 확인
import json
print(json.loads(tokenizer.to_json())['config']['word_index'])
len(tokenizer.word_index.keys())

In [ ]:
# 각 토큰과 인덱스로 구성된 딕셔너리 생성,
word_vocab = tokenizer.word_index

# <PAD> 는 0으로 추가
word_vocab['<PAD>'] = 0
vocab_size = len(tokenizer.word_index.keys())

In [ ]:
# 전체 토큰의 수 확인
import json
print(json.loads(tokenizer.to_json())['config']['word_index'])
len(tokenizer.word_index.keys())

* 문장별 토큰수에 대해 탐색적 분석을 수행해 봅시다.

In [ ]:
# 문장별 토큰 수 카운트
train_word_counts = clean_train_questions.apply(lambda x: len(x.split(' ')))
# 기초 통계량
display(pd.DataFrame(train_word_counts).describe().T)
# 분포 그래프
plt.fijure(figsize=(15, 10))
sns.histplot(train_word_counts, bins=50, label='train')
plt.legend()
plt.grid()
plt.xlabel('Number of words', fontsize=15)
plt.ylabel('Number of questions', fontsize=15)

* 문장별 토큰이 가장 큰 것이 57개 입니다. 

* 학습 입력을 위한 데이터 크기 맞추기
    * 문장이 짧기 때문에 MAX_SEQUENCE_LENGTH는 정하지 않아도 되지만,
    * 그러나 분포를 보고 적절하게 자릅시다.
    * 그리고 pad_sequences 함수를 이용하여 시퀀스데이터로 변환하기
* y는 train['type'] 와 test['type'] 입니다.

In [ ]:
MAX_SEQUENCE_LENGTH = 20

x_train = pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
x_val = pad_sequences(x_val, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [ ]:
x_train.shape

In [ ]:
y_train = train['type'].values
y_val = test['type'].values
y_train.shape, y_val.shape

#### 2) 모델링

* 토크나이징 한 데이터를 입력으로 받아 
* Embedding 레이어와 LSTM 레이어를 결합하여 이진 분류 모델링을 수행합니다.

In [ ]:
classifier_model = Sequential([
                    Embedding(input_dim=vocab_size, output_dim=64),
                    LSTM(32, return_sequences=True),
                    LSTM(16),
                    Dense(8),
                    Dense(4),
                    Dense(1, activation='sigmoid')
                    ])

In [ ]:
classifier_model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

In [ ]:
classifier_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = classifier_model.fit(x_train, y_train, callbacks=[es], validation_split=0.2, epochs=10, batch_size=32)

In [ ]:
pred = classifier_model.predict(x_val)
pred = np.where(pred >= 0.5, 1, 0)

print(accuracy_score(y_val, pred))

### (2) 사전학습모델(Word2Vec)

* Pre-trained Word2Vec model
    * 이미 다운로드 받아서 제공되었습니다.
        * ko.bin, ko.tsv
    * 참고 사이트: https://github.com/Kyubyong/wordvectors
        * 모델 파일 다운로드 사이트: https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view?resourcekey=0-Dq9yyzwZxAqT3J02qvnFwg
* 사전학습 모델을 로딩하고, 
* train 데이터셋의 질문(Q)을 임베딩벡터로 만들어, 열(Column)로 추가합니다.

#### 1) 모델 로딩
* 사전 학습된 모델을 로딩 : gensim.models.Word2Vec.load()
* 로딩 후 벡터 크기를 조회합시다. .vector_size

In [ ]:
import gensim
pre_wv_model = gensim.models.Word2Vec.load(path + 'ko.bin')

In [ ]:
import gdown
if os.path.exists(os.path.join(path, 'ko.zip')) is None:
    url = 'https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view?resourcekey=0-Dq9yyzwZxAqT3J02qvnFwg'
    output = os.path.join(path, 'ko.zip')
    gdown.download(url, output, fuzzy=True, quiet=False)
    import zipfile
    with zipfile.ZipFile(os.path.join(path, "ko.zip"), 'r') as zip_ref:
        zip_ref.extractall(path)

In [ ]:
# 모델의 벡터크기 조회
pre_mv_model.vector_size


#### 2) train 에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

In [ ]:
train['w2v'] = train['Q'].apply(lambda sent: get_sent_embedding(pre_mv_model, pre_mv_model.vector_size, tokenize('kkma', sent, join=False)))

### (3) 챗봇 구축
* 아래 절차대로 수행하는 함수 만들기
    * input 질문 
    * 1단계 : 모델을 이용하여 type 0, 1로 분류
    * 2단계 : 
        * train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
        * 코사인 유사도가 가장 높은 Q를 선택
        * 선택한 Q의 intent에 맵핑된 답변 중 하나를 무작위로 선택

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

In [ ]:
test_question = test['Q'].iloc[0]
print('질문: ', test_question)
print('답변: ', test['A'].iloc[0])
print('세부 카테고리: ' , test['intent'].iloc[0])

* 예측을 위한 입력 형태로 변환
    * 학습을 위한 전처리 과정을 test 데이터에도 적용합니다. 

In [ ]:
tokenized_sent = tokenize('mecab', test_question)
tokenized_sent_seq = tokenizer.texts_to_sequences([tokenized_sent])
tokenized_sent_input = pad_sequences(tokenized_sent_seq, amxlen=MAX_SEQUENCE_LENGTH)

* 1단계 : type 분류

In [ ]:
pred_type = classifier_model.predict(tokenized_sent_input)
pred_type = np.where(pred_type >= 0.5, 1, 0)[0][0]
pred_type

* 2단계 : 질문에 대한 벡터 만들고 코사인 유사도 계산
    * Word2Vec 사전 학습 모델로 부터 벡터 만들기

In [ ]:
test_question_w2v = get_sent_embedding(pred_wv_model, pred_wv_model.vector_size, tokenize('kkma', test_question, join=False)))

* train의 질문 벡터들과 유사도 계산
    * Word2Vec 으로 만든 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
temp_df = train[train['type'] == pred_type].copy()

temp_df['cossim_w2v'] = temp_df['w2v'].apply(lambda x: cosine_similarity([test_question]))
temp_df.head()





#### 2) 챗봇 함수 만들기
* 위 테스트 결과를 바탕으로 코드를 정리하고 함수로 생성합니다.

In [ ]:
def get_answer2(question): 
    tokenizer_sent = tokenize('mecab', question)

    tokenized_sent_seq = tokenizer.texts_to_sequences([tokenized_sent])
    tokenized_sent_input = pad_sequences(tokenized_sent_seq, maxlen=MAX_SEQUENCE_LENGTH)

    pred_type = classifier_model.predict(tokenized_sent_input)
    pred_type = np.where(pred_type >= 0.5, 1, 0)[0][0]
    test_question_s2v = get_sent_embedding(pred_wv_model, pred_wv_model.vector_size, )

    temp_df = train[train['type'] == pred_type].copy()
    temp_df['cossim_w2v'] = temp_df['w2v'].apply(lambda x: cosine_similarity([test_question]))

    intent_idx = temp_df.sort_values(['cossim_w2v'], ascending=False).head(1)['intent']
    answers = train.loc[train['train'] == intent_idx, 'A'].unique()
    answer = np.random.choice(answers)

    return answer, intent_idx

In [ ]:
question = '지원 자격이 궁금해요'
get_answer2(question)

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 4.챗봇3

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : 챗봇2의 1단계 모델을 사용합니다.
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링
- LSTM : 3-(1) 모델을 그대로 사용합니다.

### (2) FastText 모델

-  FastText 모델 학습을 위한 입력 포맷 2차원 리스트 형태 입니다.
  ```
  [['나', '는', '학생', '이다'], ['오늘', '은', '날씨', '가', '좋다']]
  ```

- Word2Vec계열의 FastText를 학습하는 이유
  - n-gram이 추가된 fasttext 모델은 유사한 단어에 대한 임베딩을 word2vec보다 잘 해결할 수 있으며, 오탈자 등에 대한 임베딩 처리가 가능하다.
  - 예) 체크카드, 쳌카드는 word2vec에서는 전혀 다른 단어이지만 fasttext는 character n-gram으로 비교적 같은 단어로 처리할 수 있다.
- 참고: https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText


#### 1) 데이터 준비
* 시작데이터 : clean_train_questions, clean_test_questions

* FastText를 위한 입력 데이터 구조 만들기

In [ ]:
fasttext_train_sentences = [x.split(' ') for x in clean_train_questions.values]

#### 2) FastText 모델 생성
* FastText 문법
    * FastText( input데이터,  min_count = , size= , window=  )
        * input데이터 : 학습에 사용할 문장으로 이루어진 리스트
        * min_count : 모델에 사용할 단어의 최소 빈도수. 이 값보다 적게 출현한 단어는 모델에 포함되지 않음. 기본값 = 5
        * size : 단어의 벡터 차원 지정. 기본값 = 100
        * window : 학습할 때 한 단어의 좌우 몇 개의 단어를 보고 예측을 할 것인지를 지정. 기본값 = 5
    * 참조 : https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText

In [ ]:
from gensim.models.fasttext import FastText
import gensim.models.word2vec
ft_model = FastText(fasttext_train_sentences, min_count=2, size=200, window=3)
print(ft_model)

#### 3) train에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

In [ ]:
train['ft_vec'] = train['Q'].apply(lambda sent: get_sent_embedding(ft_model, ft_model.vector_size, tokenize('mecab', sent, join=False)))
train.head()

### (3) 챗봇 구축
- input 질문
- intent classifier로 common와 faq 중 하나를 예측
- 예측된 intent에 속한 train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
- 코사인 유사도가 가장 높은 top-3개의 Q를 선택
- 선택한 Q에 맵핑된 답변 중 하나를 선택하고 실제 답변과 비교

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

In [ ]:
test_question = test['Q'].iloc[0]
print('질문: ', test_question)
print('답변: ', test['A'].iloc[0])
print('세부 카테고리: ' , test['intent'].iloc[0])

* 예측을 위한 입력 형태로 변환

In [ ]:
tokenized_sent = tokenize('mecab', test_question)
tokenized_sent_seq = tokenizer.texts_to_sequences([tokenized_sent])
tokenized_sent_input = pad_sequences(tokenized_sent_seq, amxlen=MAX_SEQUENCE_LENGTH)

* 예측하기

In [ ]:
pred_type = classifier_model.predict(tokenized_sent_input)
pred_type = np.where(pred_type >= 0.5, 1, 0)[0][0]
pred_type

* 질문에 대한 벡터 만들기
    * FestText 모델로 부터 벡터 만들기

In [ ]:
test_question_ft = get_sent_embedding(ft_model, ft_model.vector_size, tokenize('kkma', test_question, join=False))


* train의 질문 벡터들과 유사도 계산
    * FastText 로 만들 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

temp_df = train[train['type'] == pred_type].copy()

temp_df['cossim_ft'] = temp_df['ft_vec'].apply(lambda x: cosine_similarity([test_question]))
temp_df.head()




#### 2) 함수로 생성하기

In [ ]:
def get_answer3(question): 
    tokenizer_sent = tokenize('mecab', question)

    tokenized_sent_seq = tokenizer.texts_to_sequences([tokenized_sent])
    tokenized_sent_input = pad_sequences(tokenized_sent_seq, maxlen=MAX_SEQUENCE_LENGTH)

    pred_type = classifier_model.predict(tokenized_sent_input)
    pred_type = np.where(pred_type >= 0.5, 1, 0)[0][0]
    test_question_s2v = get_sent_embedding(pred_wv_model, pred_wv_model.vector_size, )

    temp_df = train[train['type'] == pred_type].copy()
    temp_df['cossim_ft'] = temp_df['ft_vec'].apply(lambda x: cosine_similarity([test_question]))

    intent_idx = temp_df.sort_values(['cossim_ft'], ascending=False).head(3).index
    choiced-idx = np.random.choice(intent_idx)
    answer = train.loc[choiced_idx, 'A']

    return answer, train.loc[choiced_idx, 'intent']

In [ ]:
question = '나이 제한이 있나요'
get_answer3(question)

In [ ]:
ft_model.wv.similar_by_word['나이']

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

In [ ]:
ct = 0
for idx, row in test.iterrows():
    print('질문: ', row['Q'])
    answer, intext_idx = get_answer3(row['Q'])
    print('예상 답변: ', answer)
    if intent_idx == row['intent']:
        ct += 1
print(ct, test.shape[0])

## 5.질문에 대한 답변 비교해보기

* **세부요구사항**
    * 세가지 챗봇을 생성해 보았습니다. 
    * 질문을 입력하여 답변을 비교해 봅시다. 어떤 챗봇이 좀 더 정확한 답변을 하나요?
